## Загрузка данных

In [ ]:
import json
import pandas as pd
import re
import spacy
import nltk
from nltk.corpus import stopwords
from rank_bm25 import BM25Okapi
import tqdm
import numpy as np

nltk.download('stopwords')
nlp = spacy.load("ru_core_news_sm", disable=['parser', 'ner'])

In [ ]:
with open('fun_train.json', 'r', encoding='utf-8') as f:
  data = json.load(f)

In [ ]:
len(data)

251416

In [ ]:
df = pd.DataFrame(data.items(), columns=['text', 'is_joke'])

In [ ]:
df

,text,is_joke
0,"Нам нужно расстаться. Пойми, дело не в тебе......",1
1,Где твоя грудь? Ты ее спугнул.,1
2,"Разбивая кому-то сердце, знай, у него есть еще...",1
3,"Если бы я была принцессой, то моим замком был ...",1
4,Давай ты потратишь на меня лучшие годы своей ж...,1
...,...,...
251411,"Сильно хочется в горы! Некоторый опыт есть,сна...",0
251412,"Пусть глава РЖД Якунин ездит не дальше, чем ро...",1
251413,А как с бабушкой в Питере получилось? Почему е...,0
251414,Останавливает инспектор ГАИ машину: права есть...,1


In [ ]:
with open('fun_test.json', 'r', encoding='utf-8') as f:
  data_ = json.load(f)

In [ ]:
df_ = pd.DataFrame(data_.items(), columns=['text', 'is_joke'])

In [ ]:
df_

,text,is_joke
0,"Я весь день, как «Глагол»! А ты, всё время «Не...",1
1,Скупой платит дважды. Пойду работать к скупому.,1
2,"Алло. Нет, милая, конечно, не сплю, сегодня же...",1
3,"У любой женщины должна быть подруга- уебище, к...",1
4,Сходи со мной в ларек.,1
...,...,...
61789,"Не люблю салажат, которые только и могут, что ...",0
61790,Да откуда же такие сволочи на земле берутся???...,0
61791,Эй эй не стоит напиваться не последний день жи...,0
61792,"А я еще думала, как бы так намекнуть, чтоб поп...",0


## Предобработка данных

In [ ]:
def preprocessing(texts_df):

  # удаление электронных почт
  texts_df['clean_text'] = texts_df['text'].apply(lambda x: re.sub(r"([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)", '', x))

  # удаление хэштегов, в которых после символа # идет больше 7 символов
  texts_df['clean_text'] = texts_df['clean_text'].apply(lambda x: re.sub(r"#([^\s]{7,})", '', x))

  # удаление пунктуации
  punctuation = '.,\'!"#$%&()*+-:;<=>?@[\\]^_`{|}~/'
  texts_df['clean_text'] = texts_df['clean_text'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

  # удаление чисел
  texts_df['clean_text'] = texts_df['clean_text'].apply(lambda x: re.sub(r"[0-9]", ' ', x))

  # удаление пробелов
  texts_df['clean_text'] = texts_df['clean_text'].apply(lambda x:' '.join(x.split()))

  # приведение к нижнему регистру
  texts_df['clean_text'] = texts_df['clean_text'].str.lower()

  return texts_df

In [ ]:
df = preprocessing(df)
df_ = preprocessing(df_)

In [ ]:
def delete_stopwords(data):

  output = []
  for text in data['clean_text']:
    s = [token for token in text.split() if not token in stopwords.words('russian')]
    text = ' '.join(s)
    out = [token.lemma_ for token in nlp(text)]
    output.append(' '.join(out))

  return output

In [ ]:
df['lemmas'] = delete_stopwords(df)
df_['lemmas'] = delete_stopwords(df_)

In [ ]:
df = df[df.lemmas != '']
df_ = df_[df_.lemmas != '']

## Уменьшение датасета

In [ ]:
df_1 = df.loc[df['is_joke'] == 1]
df_0 = df.loc[df['is_joke'] == 0]

jokes = df_1.sample(n=15000)

In [ ]:
tokenized_corpus = [text.split() for text in df_0['lemmas']]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
jokes_ = [joke.split() for joke in jokes['lemmas']]

In [ ]:
used = []

for joke in tqdm.tqdm(jokes_):
  variants = bm25.get_top_n(joke, list(df_0['lemmas']), n=3)
  for variant in variants:
    if list(df_0['lemmas']).index(variant) not in used:
      used.append(list(df_0['lemmas']).index(variant))
      break
    else:
      continue

In [ ]:
df_0['id'] = np.arange(len(df_0))[::1]

In [ ]:
notjokes = pd.DataFrame()

for i in used:
  nj = df_0.loc[df_0['id'] == i]
  notjokes = pd.concat([notjokes, nj], axis=0)

In [ ]:
train = pd.concat([jokes, notjokes], axis=0)
train.to_csv('tarin_30k.csv', sep='\t', index=False)